**Installing necessary Libraries**

In [ ]:
!pip install insightface
!pip install diffusers
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 96.3 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1055345 sha256=45b161dd58fed2a53843d51c0c00be8b5ae613c608f06323cdc558f1670fcdb0
  Stored in directory: /root/.cache/pip/wheels/e3/d0/80/e3773fb8b6d1cca87ea1d33d9b1f20a223a6493c896da249b5
Successfully built insightface
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00


# **Step 1 : Processing the Images**

**Imprting the required Libraries**

In [ ]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image

import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

**Initializing the app object for face detection**

In [ ]:
app = FaceAnalysis(name='buffalo_l') # enable detection model only
app.prepare(ctx_id=0, det_size=(640, 640))

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 83648.98KB/s]
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


**Initializing the face swapper object**

In [ ]:
swapper =  insightface.model_zoo.get_model('/content/drive/MyDrive/inswapper_128.onnx',
                                           download = False,
                                           download_zip = False)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
inswapper-shape: [1, 3, 128, 128]


In [ ]:
def swap_faces(img_1, img_2, app, swapper, plot_before=False, plot_after=True):
  img1 = cv2.imread(img_1)
  img2 = cv2.imread(img_2)
  if plot_before:
    fig, axs = plt.subplots(1, 2, figsize=(10, 8))
    axs[0].imshow(img1[:,:,::-1])
    axs[0].axis('off')

    axs[1].imshow(img2[:,:,::-1])
    axs[1].axis('off')

    plt.show()

  #Identify the faces
  face1 = app.get(img1)
  face2 = app.get(img2)

  img1_copy = img1.copy()
  img2_copy = img2.copy()

  if plot_after:
    img1_copy = swapper.get(img1_copy, face1[0], face2[0], paste_back=True)
    img2_copy = swapper.get(img2_copy, face2[0], face1[0], paste_back=True)

    '''
    fig, axs = plt.subplots(1, 2, figsize=(10, 8))
    axs[0].imshow(img1_copy[:,:,::-1])
    axs[0].axis('off')

    axs[1].imshow(img2_copy[:,:,::-1])
    axs[1].axis('off')

    plt.show()'''

  # Save images
  # Extract filename and extension from image paths
  img1_filename, img1_ext = os.path.splitext(os.path.basename(img_1))
  #img2_filename, img2_ext = os.path.splitext(os.path.basename(img_2))

  # Create custom save paths with "swapped_" prefix
  img1_save_path = os.path.join(os.path.dirname(img_1), f"swapped_{img1_filename}{img1_ext}")
  #img2_save_path = os.path.join(os.path.dirname(img_2), f"swapped_{img2_filename}{img2_ext}")

  # Save swapped images
  cv2.imwrite(img1_save_path, img1_copy)
  #cv2.imwrite(img2_save_path, img2_copy)

  return img1_save_path

In [ ]:
def merge_images(image_path1, image_path2, output_path="output_image.jpg"):
  # Open the images
  image1 = Image.open(image_path1)
  image2 = Image.open(image_path2)

  # Resize images if necessary
  width = max(image1.width, image2.width)
  height = max(image1.height, image2.height)
  image1 = image1.resize((width, height))
  image2 = image2.resize((width, height))

  # Create a new blank image with the combined dimensions
  merged_image = Image.new("RGB", (width * 2, height))

  # Paste the first image on the left
  merged_image.paste(image1, (0, 0))

  # Paste the second image on the right
  merged_image.paste(image2, (width, 0))

  # Save the merged image
  merged_image.save(output_path)

  return output_path

# **Function for creating a image to video**

**Import Required Libraries**

In [ ]:
import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Load the model
pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt",
    torch_dtype=torch.float16,
    variant="fp16"
)
pipe.enable_model_cpu_offload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/984 [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/196M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/3.05G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

# **Processing the image**

In [ ]:
Child_Image = swap_faces('/content/Child.jpg','/content/Daniel_Child.jpg',app, swapper)

/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [ ]:
Adult_Image = swap_faces('/content/Adult.jpg','/content/Daniel_Adult.jpg',app, swapper)

In [ ]:
output_image = merge_images(Child_Image, Adult_Image)

**Generate Video**

In [ ]:
# Load an input image
image = load_image(output_image)
image = image.resize((1024, 576))

# Generate video frames
generator = torch.manual_seed(42)
frames = pipe(image, decode_chunk_size=8, generator=generator).frames[0]

# Save the video
export_to_video(frames, "generated_video.mp4", fps=7)

  0%|          | 0/25 [00:00<?, ?it/s]

'generated_video.mp4'